In [1]:
import numpy as np
import pandas as pd
import json

# Word Bank

## We need to aggregate the data

In [2]:
wb_data = pd.read_csv("data/wb_out_FULL.csv")
wb_data

,Country Name,Country Code,Indicator Name,Indicator Code,Most Recent Value,Year,GID
0,Afghanistan,AFG,Access to electricity (% of population),EG.ELC.ACCS.ZS,9.770000e+01,2017.0,AFG
1,Afghanistan,AFG,Adequacy of social insurance programs (% of to...,per_si_allsi.adq_pop_tot,1.569503e+01,2007.0,AFG
2,Afghanistan,AFG,"Adjusted net enrollment rate, primary (% of pr...",SE.PRM.TENR,2.676600e+01,1993.0,AFG
3,Afghanistan,AFG,"Adjusted net enrollment rate, primary, female ...",SE.PRM.TENR.FE,1.369764e+01,1993.0,AFG
4,Afghanistan,AFG,"Adjusted net enrollment rate, primary, male (%...",SE.PRM.TENR.MA,3.879503e+01,1993.0,AFG
...,...,...,...,...,...,...,...
14185,Zimbabwe,ZWE,Surface area (sq. km),AG.SRF.TOTL.K2,3.907600e+05,2018.0,ZWE
14186,Zimbabwe,ZWE,Urban population,SP.URB.TOTL,4.650663e+06,2018.0,ZWE
14187,Zimbabwe,ZWE,Urban population (% of total population),SP.URB.TOTL.IN.ZS,3.220900e+01,2018.0,ZWE
14188,Zimbabwe,ZWE,Urban population growth (annual %),SP.URB.GROW,1.323892e+00,2018.0,ZWE


In [3]:
wb_data["Most Recent Value"] = wb_data["Most Recent Value"].fillna("")

In [4]:
properties = []
for idx, row in wb_data.iterrows():
    properties.append({row["Indicator Code"]: {"Indicator Name": row["Indicator Name"],
                                               "Most Recent Value": row["Most Recent Value"],
                                               "Year": row["Year"]
    }})

wb_data["properties"] = properties

In [5]:
wb_data.drop(["Indicator Code", "Indicator Name", "Most Recent Value", "Year"], axis=1, inplace=True)

In [6]:
def merge_list_of_dict(l):
    d = {}
    for elem in l:
        d.update(elem)
    return d

In [7]:
wb_data_aggr = wb_data.groupby(["Country Name", "Country Code", "GID"]).agg({'properties':lambda x: merge_list_of_dict(list(x))}).reset_index()

In [8]:
# convert the GID to list
wb_data_aggr.GID = wb_data_aggr.GID.apply(lambda x: [x])
# rename some columns
wb_data_aggr = wb_data_aggr.rename(columns={"GID": "gid",
                                            "properties": "Properties",
                                            "Country Code":"Country ISO3"})
wb_data_aggr

,Country Name,Country ISO3,gid,Properties
0,Afghanistan,AFG,[AFG],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...
1,Albania,ALB,[ALB],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...
2,Algeria,DZA,[DZA],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...
3,American Samoa,ASM,[ASM],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...
4,Andorra,AND,[AND],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...
...,...,...,...,...
210,Virgin Islands (U.S.),VIR,[VIR],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...
211,West Bank and Gaza,PSE,[PSE],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...
212,"Yemen, Rep.",YEM,[YEM],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...
213,Zambia,ZMB,[ZMB],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...


In [21]:
# add source, year and samplesize column
wb_data_aggr["Source"] = "WB"
wb_data_aggr["Year"] = 2020
wb_data_aggr["Sample Size"] = -1
wb_data_aggr["adm level"] = 0
wb_data_aggr

,Country Name,Country ISO3,gid,Properties,Source,Year,Sample Size,adm level
0,Afghanistan,AFG,[AFG],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
1,Albania,ALB,[ALB],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
2,Algeria,DZA,[DZA],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
3,American Samoa,ASM,[ASM],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
4,Andorra,AND,[AND],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
...,...,...,...,...,...,...,...,...
210,Virgin Islands (U.S.),VIR,[VIR],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
211,West Bank and Gaza,PSE,[PSE],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
212,"Yemen, Rep.",YEM,[YEM],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
213,Zambia,ZMB,[ZMB],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0


# IVS_country

In [22]:
IVS_country = pd.read_pickle("data/IVS_country_GID.pkl")
IVS_country = IVS_country.rename(columns={"Survey": "Source",
                                          "Wave": "Year"})
IVS_country.drop("Country Numeric", axis=1, inplace=True)
IVS_country["adm level"] = 0
IVS_country

,Source,Year,Country Name,Country ISO3,Sample Size,gid,Properties,adm level
0,EVS,1981-1984,Belgium,BEL,1145,[BEL],{'A001': {'Label': 'Important in life: Family'...,0
1,EVS,1981-1984,Canada,CAN,1254,[CAN],{'A001': {'Label': 'Important in life: Family'...,0
2,EVS,1981-1984,Denmark,DNK,1182,[DNK],{'A001': {'Label': 'Important in life: Family'...,0
3,EVS,1981-1984,France,FRA,1200,[FRA],{'A001': {'Label': 'Important in life: Family'...,0
4,EVS,1981-1984,Germany,DEU,1305,[DEU],{'A001': {'Label': 'Important in life: Family'...,0
...,...,...,...,...,...,...,...,...
361,WVS,2010-2014,Egypt,EGY,1523,[EGY],{'A001': {'Label': 'Important in life: Family'...,0
362,WVS,2010-2014,United States of America (the),USA,2232,[USA],{'A001': {'Label': 'Important in life: Family'...,0
363,WVS,2010-2014,Uruguay,URY,1000,[URY],{'A001': {'Label': 'Important in life: Family'...,0
364,WVS,2010-2014,Uzbekistan,UZB,1500,[UZB],{'A001': {'Label': 'Important in life: Family'...,0


# IVS_region

In [23]:
IVS_region = pd.read_pickle("data/IVS_region_GID.pkl")
IVS_region = IVS_region.rename(columns={"Survey": "Source",
                                          "Wave": "Year"})
IVS_region.drop("Country Numeric", axis=1, inplace=True)
IVS_region["adm level"] = 1
IVS_region

,Source,Year,Country Name,Country ISO3,Sample Size,gid,Properties,adm level
0,WVS,1999-2004,Bangladesh,BGD,199,[BGD.3.1_1],{'original region code': {'Region Numeric': 50...,1
1,WVS,1999-2004,Bangladesh,BGD,121,[BGD.2.4_1],{'original region code': {'Region Numeric': 50...,1
2,WVS,1999-2004,Bangladesh,BGD,105,[BGD.1.2_1],{'original region code': {'Region Numeric': 50...,1
3,WVS,1999-2004,Bangladesh,BGD,128,[BGD.3.10_1],{'original region code': {'Region Numeric': 50...,1
4,WVS,1999-2004,Bangladesh,BGD,81,[BGD.3.17_1],{'original region code': {'Region Numeric': 50...,1
...,...,...,...,...,...,...,...,...
470,EVS,2008-2010,United Kingdom of Great Britain and Northern I...,GBR,22,"[GBR.4.13_1, GBR.4.15_1, GBR.4.5_1, GBR.4.21_1...",{'original region code': {'Region Numeric': -5...,1
471,EVS,2008-2010,United Kingdom of Great Britain and Northern I...,GBR,43,"[GBR.3.3_1, GBR.3.5_1, GBR.3.7_1, GBR.3.10_1, ...",{'original region code': {'Region Numeric': -5...,1
472,EVS,2008-2010,United Kingdom of Great Britain and Northern I...,GBR,43,"[GBR.3.4_1, GBR.3.6_1, GBR.3.8_1, GBR.3.9_1, G...",{'original region code': {'Region Numeric': -5...,1
473,EVS,2008-2010,United Kingdom of Great Britain and Northern I...,GBR,6,"[GBR.3.4_1, GBR.3.13_1, GBR.3.17_1, GBR.3.20_1...",{'original region code': {'Region Numeric': -5...,1


# Unify the data

In [24]:
country_statistics = pd.concat([wb_data_aggr, IVS_country, IVS_region], ignore_index=True)
country_statistics.to_pickle("to_upload/country_statistics.pkl")
country_statistics

,Country Name,Country ISO3,gid,Properties,Source,Year,Sample Size,adm level
0,Afghanistan,AFG,[AFG],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
1,Albania,ALB,[ALB],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
2,Algeria,DZA,[DZA],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
3,American Samoa,ASM,[ASM],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
4,Andorra,AND,[AND],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
...,...,...,...,...,...,...,...,...
1043,United Kingdom of Great Britain and Northern I...,GBR,"[GBR.4.13_1, GBR.4.15_1, GBR.4.5_1, GBR.4.21_1...",{'original region code': {'Region Numeric': -5...,EVS,2008-2010,22,1
1044,United Kingdom of Great Britain and Northern I...,GBR,"[GBR.3.3_1, GBR.3.5_1, GBR.3.7_1, GBR.3.10_1, ...",{'original region code': {'Region Numeric': -5...,EVS,2008-2010,43,1
1045,United Kingdom of Great Britain and Northern I...,GBR,"[GBR.3.4_1, GBR.3.6_1, GBR.3.8_1, GBR.3.9_1, G...",{'original region code': {'Region Numeric': -5...,EVS,2008-2010,43,1
1046,United Kingdom of Great Britain and Northern I...,GBR,"[GBR.3.4_1, GBR.3.13_1, GBR.3.17_1, GBR.3.20_1...",{'original region code': {'Region Numeric': -5...,EVS,2008-2010,6,1


# Create table and upload the data

In [25]:
import pandas as pd
import json
import psycopg2

In [26]:
def create_country_stats_table(cur):
    sql_create_country_stats_table = """ 
        CREATE TABLE IF NOT EXISTS country_statistics (
            source text NOT NULL,
            year text NOT NULL,
            country text NOT NULL,
            countrycode text NOT NULL, 
            adm_level text NOT NULL,
            gid text[] NOT NULL,
            samplesize int,
            properties jsonb NOT NULL,
            UNIQUE (gid, source, year, adm_level),
            PRIMARY KEY (gid, source, year, adm_level)
        )"""

    cur.execute(sql_create_country_stats_table)

    print("country_statistics table created")

In [27]:
def drop_country_stats_table(cur):
    sql_drop_country_stats_table = """DROP TABLE country_statistics"""
    
    try:
        cur.execute(sql_drop_country_stats_table)
        print("country_statistics table deleted")
    except:
        print("country_statistics not present")

In [28]:
def import_data(cur,data):
    
    INSERT_STATEMENT = 'INSERT INTO country_statistics \
                            (source, year, \
                             country, countrycode,  \
                             adm_level, \
                             gid, samplesize, \
                             properties) \
                        VALUES (%s, %s, \
                                %s, %s, \
                                %s, \
                                %s, %s, \
                                %s);'
    

    for idx, row in data.iterrows():
        cur.execute(INSERT_STATEMENT, (row['Source'], row['Year'],
                                       row['Country Name'], row['Country ISO3'], 
                                       row['adm level'],
                                       row['gid'], row['Sample Size'], 
                                       json.dumps(row["Properties"])))

In [29]:
# Connect to covid19db.org
conn = psycopg2.connect(
    host='covid19db.org',
    port=5432,
    dbname='covid19db_adm_play',
    user='covid19db_adm_rw',
    password='fGt962FdeG2yXj3c4d3'
)

cur = conn.cursor()

In [30]:
# loads data from csv
country_stats = pd.read_pickle("to_upload/country_statistics.pkl")
type(country_stats.gid[0])

list

In [31]:
drop_country_stats_table(cur)
conn.commit()

create_country_stats_table(cur)
conn.commit()

import_data(cur, country_statistics)
conn.commit()
print("data uploaded")


country_statistics table deleted
country_statistics table created
data uploaded


# TEST

In [32]:
# Connect to covid19db.org
conn = psycopg2.connect(
    host='covid19db.org',
    port=5432,
    dbname='covid19db_adm_play',
    user='covid19db_adm_rw',
    password='fGt962FdeG2yXj3c4d3'
)

cur = conn.cursor()

In [33]:
cur.execute("""SELECT * FROM country_statistics""")

data = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
data


,source,year,country,countrycode,adm_level,gid,samplesize,properties
0,WB,2020,Afghanistan,AFG,0,[AFG],-1,"{'EN.POP.DNST': {'Year': 2018.0, 'Indicator Na..."
1,WB,2020,Albania,ALB,0,[ALB],-1,"{'EN.POP.DNST': {'Year': 2018.0, 'Indicator Na..."
2,WB,2020,Algeria,DZA,0,[DZA],-1,"{'EN.POP.DNST': {'Year': 2018.0, 'Indicator Na..."
3,WB,2020,American Samoa,ASM,0,[ASM],-1,"{'EN.POP.DNST': {'Year': 2018.0, 'Indicator Na..."
4,WB,2020,Andorra,AND,0,[AND],-1,"{'EN.POP.DNST': {'Year': 2018.0, 'Indicator Na..."
...,...,...,...,...,...,...,...,...
1043,EVS,2008-2010,United Kingdom of Great Britain and Northern I...,GBR,1,"[GBR.4.13_1, GBR.4.15_1, GBR.4.5_1, GBR.4.21_1...",22,{'A001': {'Label': 'Important in life: Family'...
1044,EVS,2008-2010,United Kingdom of Great Britain and Northern I...,GBR,1,"[GBR.3.3_1, GBR.3.5_1, GBR.3.7_1, GBR.3.10_1, ...",43,{'A001': {'Label': 'Important in life: Family'...
1045,EVS,2008-2010,United Kingdom of Great Britain and Northern I...,GBR,1,"[GBR.3.4_1, GBR.3.6_1, GBR.3.8_1, GBR.3.9_1, G...",43,{'A001': {'Label': 'Important in life: Family'...
1046,EVS,2008-2010,United Kingdom of Great Britain and Northern I...,GBR,1,"[GBR.3.4_1, GBR.3.13_1, GBR.3.17_1, GBR.3.20_1...",6,{'A001': {'Label': 'Important in life: Family'...
